ChaoTang


# Practical 7 - Part 2A
This project explores the geometry of a single camera. The aim is to take several points on
a plane, and predict where they will appear in the camera image. Based on these observed
points, we will then try to re-estimate the Euclidean transformation relating the plane and
the camera. In practical 2B we will use this code to draw a wireframe cube
on an augmented reality marker.   You should use this
template for your code and fill in the missing sections marked "TO DO"

# Import libraries 

In [4]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# TO DO - fill in projectiveCamera and estimatePlanePose functions  (you will have to utilise your solutions for solveAXEqualsZero and calcBestHomography from Part 1)

In [5]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0   
    U,L,VT = np.linalg.svd(A)
    VRow,Vcol = VT.shape
    h = VT[VRow -1,:]
    return h

In [6]:
# This function should apply the direct linear transform (DLT) algorithm to calculate the best 
# homography that maps the points in pts1Cart to their corresonding matching in pts2Cart
def calcBestHomography(pts1Cart, pts2Cart):    
    # TO DO: replace this
    # H = 
    
    # TO DO: 
    # First convert points into homogeneous representation
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    pts2Hom = np.concatenate((pts2Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    
    # Then construct the matrix A, size (2 * n_points,9)
    n_point, = pts1Cart[0,:].shape
    A = np.zeros((2 * n_point,9))
    
    for cpoint in range(n_point):
        currentVec1 = pts1Hom[:,cpoint]
        currentVec2 = pts2Hom[:,cpoint]
        
        A[2 * cpoint,:] = [0,0,0,-currentVec1[0],-currentVec1[1],-1,currentVec1[0]*currentVec2[1],currentVec1[1]*currentVec2[1],currentVec2[1]]
        A[2 * cpoint+1,:] = [currentVec1[0],currentVec1[1],1,0,0,0,-currentVec2[0]*currentVec1[0],-currentVec2[0]*currentVec1[1],-currentVec2[0]]
    
    # Solve Ah = 0
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    H = np.reshape(h,(3,3))
    return H

In [7]:
#The goal of this function is to project points in XCart through projective camera
#defined by intrinsic matrix K and extrinsic matrix T.
def projectiveCamera(K,T,XCart):
    
    # TO DO: Replace this
    # XImCart =
    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom 
    ndim,ndata = XCart.shape
    hzero = np.ones(ndata)
    XHom = np.vstack((XCart,hzero))
    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    FCamera = T @ XHom
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    xCamHom = FCamera[0:3,:]
    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    xImHom = K @ xCamHom
    # TO DO: Convert points back to Cartesian coordinates xImCart
    XImCart_old = xImHom / xImHom[2,:]
    XImCart = XImCart_old[0:2,:]
    
    return XImCart


In [13]:
# Goal of function is to estimate pose of plane relative to camera (extrinsic matrix)
# given points in image xImCart, points in world XCart and intrinsic matrix K

def estimatePlanePose(XImCart,XCart,K):

    # TO DO: replace this
    # T = 

    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    ndim,ndata = XImCart.shape
    hzero = np.ones(ndata)
    XImHom = np.vstack((XImCart,hzero))
    
    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.linalg.inv(K) @ XImHom
    
    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    H = calcBestHomography(XCart[0:2,:],XCamHom)
          
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD       
    A = H[:,0:2]
    U,L,VT = np.linalg.svd(A)
    R = np.zeros((3,3))
    Temp = np.array([[1,0],[0,1],[0,0]])
    R[:,0:2] = U @Temp @ VT
    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R[:,2] = np.cross(R[:,0],R[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    Condition = np.linalg.det(R)
    if Condition <= 0:
        R[:,2] = -R[:,2]
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    k = 0
    for m in range(3):
        for n in range(2):
            k = k + H[m,n]/R[m,n]
            print('m,n = ',m,n)
    k = k/6
    print('k = ',k)
    Tao = np.zeros((3,1))
    Tao[:,0] = H[:,2]/k
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if Tao[2,:] <0:
        Tao = -Tao
        R[:,0:2] = -R[:,0:2]
    # TO DO: Assemble transformation into matrix form
    T = np.hstack((R,Tao))
    print(T)
    
    return T 

# Once you have completed these functions, use them to estimate the transformation from the plane co-ordinate system to the camera co-ordinate system (i.e. the extrinsic matrix)

In [16]:
# We assume that the intrinsic camera matrix K is known and has values
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are four known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])


# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])

# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera" - body of function appears below:

XImCart = projectiveCamera(K,T,XCart)
print(XImCart)
# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
ndim,ndata = XImCart.shape
for iteration in range(ndata):
    XImCart[:,iteration] = XImCart[:,iteration] + np.random.normal(0,1,ndim)

# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function "estimate plane pose"
TEst = estimatePlanePose(XImCart,XCart,K)
# If you have got this correct, Test should closely resemble T above

[[267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
 [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]]
m,n =  0 0
m,n =  0 1
m,n =  1 0
m,n =  1 1
m,n =  2 0
m,n =  2 1
k =  0.0019828500731162006
[[ 9.85321598e-01 -4.91760877e-02  1.63471896e-01  4.60552660e+01]
 [-1.63061346e-01 -5.54529578e-01  8.16031828e-01  6.96478752e+01]
 [ 5.05207490e-02 -8.30709732e-01 -5.54408689e-01  4.97362427e+02]]


## Comments:
we used the points coordinates in image,points coordinates in world and intrinsic matrix K to estimate the extrinsic matrix here.We can see the errors between the estimate result and the reference T are small.